In [10]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12,4


In [3]:
def modelfit(alg,dtrain,dtest,predictiors, performCV = True, printFeatureImportance=True, cv_folds = 5):
    alg.fit(dtrain[predictors],dtrain['Is_Shortlisted'])
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    if performCV:
        cv_score = cross_validation.cross_val_score(alg,dtrain[predictors],dtrain['Is_Shortlisted'], cv = cv_folds, scoring = 'roc_auc')
        print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g"%(np.mean(cv_score),np.std(cv_score),np.min(cv_score),
                                                                        np.max(cv_score))
    print "Accuracy - %.4g" % metrics.accuracy_score(dtrain['Is_Shortlisted'],dtrain_predictions)
    print "AUC Score - %f" % metrics.roc_auc_score(dtrain['Is_Shortlisted'],dtrain_predprob)
    
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_,predictors).sort_values(ascending=False)
        feat_imp.plot(kind = 'bar', title = 'Feature Importance')
        plt.ylabel('Feature Importance Score')


In [4]:
data = pd.read_csv("C:/Users/user/Documents/data3.csv")
internship = pd.read_csv("C:/Users/user/Documents/Internship3.csv")
student = pd.read_csv("C:/Users/user/Documents/Student3.csv")
student_exp = pd.read_csv("C:/Users/user/Documents/Student_Exp3.csv")

In [54]:
#data.apply(lambda x: sum(x.isnull()))
#internship.apply(lambda x : sum(x.isnull()))
#student.apply(lambda x : sum(x.isnull()))
#student_exp.apply(lambda x : sum(x.isnull()))
#datais.dtypes
dataise.apply(lambda x: sum(x.isnull()))
#student_exp.apply(lambda x : sum(x.isnull()))
#student_exp.head()
#student_exp_unique.head()
#dataise.shape

Internship_ID                       0
Student_ID                          0
Earliest_Start_Date                 0
Expected_Stipend                    0
Minimum_Duration                    0
Preferred_location                  0
Is_Part_Time                        0
Is_Shortlisted                 107428
source                              0
Internship_Profile                  0
Skills_required                     0
Internship_Type                     0
Internship_Location                 0
Internship_category                 0
No_of_openings                      0
Stipend_Type                        0
Stipend1                            0
Internship_deadline                 0
Start_Date                          0
Internship_Duration.Months.         0
skill_compress_str                  0
Unnamed: 0                          0
Institute_Category                  0
Degree                             17
Stream                            123
Current_year                        0
Year_of_grad

In [5]:
data['Preferred_location'] = data['Preferred_location'].apply(lambda x : 'NS' if pd.isnull(x) else x)
internship['Skills_required'] = internship['Skills_required'].apply(lambda x : 'NS' if pd.isnull(x) else x)
internship['skill_compress_str'] = internship['skill_compress_str'].apply(lambda x : 'NS' if pd.isnull(x) else x)
datai = pd.merge(data,internship,how = 'left', on = 'Internship_ID')
datais = pd.merge(datai,student,how = 'left', on = 'Student_ID')
datais['Stipend1']= datais['Stipend1'].fillna(0)
datais_dates = datais[['Student_ID','Earliest_Start_Date']]
student_exp['Experience_Type'] = student_exp['Experience_Type'].fillna('NE')
student_exp['Profile'] = student_exp['Profile'].fillna('NS')
student_exp['exp_days'] = 0
student_exp = student_exp[student_exp['Start.Date'].notnull()]
student_exp['Profile'] = student_exp[['Student_ID','Profile']].groupby(['Student_ID'])['Profile'].transform(lambda x: ','.join(x))
student_exp['Experience_Type'] = student_exp[['Student_ID','Experience_Type']].groupby(['Student_ID'])['Experience_Type'].transform(lambda x: ','.join(x))
student_exp_unique = student_exp[['Student_ID','Experience_Type','Profile']].drop_duplicates()
dataise = pd.merge(datais,student_exp_unique,on = 'Student_ID')



In [6]:
dataise['avg_stipend'] = 0
dataise['avg_stipend'] = np.where(dataise['Stipend2'].isnull(),dataise['Stipend1'],(dataise['Stipend1']+dataise['Stipend2'])/2)
dataise.drop('Stipend2',axis=1,inplace=True)
dataise['percentage_pg'] = np.where(dataise['PG_scale'] == 4,(dataise['Performance_PG']+1)*20,np.where(dataise['PG_scale'] ==10 ,
                                                                                                   dataise['Performance_PG']*10,
                                                                                                   dataise['Performance_PG']))
dataise['percentage_ug'] = np.where(dataise['UG_Scale'] == 4,(dataise['Performance_UG']+1)*20,np.where(dataise['UG_Scale'] ==10 ,
                                                                                                   dataise['Performance_UG']*10,
                                                                                                   dataise['Performance_UG']))
dataise['student_stipend'] = np.where(dataise['Expected_Stipend']=='No Expectations',0 ,np.where(dataise['Expected_Stipend']==
                                                                                     '2-5K',2000,np.where(dataise['Expected_Stipend']==
                                                                                                         '5-10K',5000,10000)))
dataise['Stipend_low_match'] = np.where(dataise['Stipend1'] >= dataise['student_stipend'],'Yes','No')
dataise['Stipend_avg_match'] = np.where(dataise['avg_stipend'] >= dataise['student_stipend'],'Yes','No')
dataise.drop('Performance_PG',axis=1,inplace = True)
dataise.drop('Performance_UG',axis=1,inplace = True)
dataise.drop('Unnamed: 0',axis=1,inplace = True)
#dataise.drop('Unnamed: 0_x',axis=1,inplace = True)
#dataise.drop('Unnamed: 0_y',axis=1,inplace = True)
#dataise.drop('Start_Date',axis=1,inplace = True)
dataise['location_match'] = np.where(dataise['Preferred_location']=='NS','Yes',np.where(dataise['Preferred_location']==
                                                                                     dataise['Internship_Location'],'Yes','No'))
#dataise.drop('Preferred_location',axis=1,inplace=True)
#dataise.drop('Internship_Location',axis=1,inplace=True)

#dataise.drop('Expected_Stipend',axis=1,inplace=True)


In [60]:
#datais['avg_stipend'].head()
#datais['percentage_pg'][:50]
#datais['PG_scale'].value_counts()
#datais['percentage_pg'].value_counts()
#datais['Performance_PG'].value_counts()
#datais['percentage_ug'].value_counts()
#datais['location_match'].value_counts()
#data_new.head()
#datais['Stipend_low_match'].value_counts()
#datais['Stipend_avg_match'].value_counts()
#datais['Expected_Stipend'].value_counts()
#datais['student_stipend'].value_counts()
#datais.to_csv("semi_final_data.csv")
#datais['Internship_Duration.Months.'].value_counts()
#datais['Minimum_Duration'].value_counts()
dataise.dtypes

Internship_ID                    int64
Student_ID                       int64
Earliest_Start_Date             object
Expected_Stipend                object
Minimum_Duration                 int64
Preferred_location              object
Is_Part_Time                     int64
Is_Shortlisted                 float64
source                          object
Internship_Profile              object
Skills_required                 object
Internship_Type                 object
Internship_Location             object
Internship_category             object
No_of_openings                   int64
Stipend_Type                    object
Stipend1                       float64
Internship_deadline             object
Start_Date                      object
Internship_Duration.Months.      int64
skill_compress_str              object
Institute_Category              object
Degree                          object
Stream                          object
Current_year                    object
Year_of_graduation       

In [7]:
dataise['Internship_Profile'] = [nltk.word_tokenize(sent) for sent in dataise['Internship_Profile']]
dataise['Skills_required'] = [nltk.word_tokenize(sent) for sent in dataise['Skills_required']]
dataise['skill_compress_str'] = [nltk.word_tokenize(sent) for sent in dataise['skill_compress_str']]
dataise['Profile'] = [nltk.word_tokenize(sent.decode('utf8', 'ignore')) for sent in dataise['Profile']]

In [89]:
#dataise['Degree_base'] = 'Yes'
dataise['Degree_base'] = np.where(re.search(r'^B[a-z]+',str(dataise['Degree'])),'UG','No')
dataise['Degree_base'].value_counts()

No    300010
Name: Degree_base, dtype: int64

In [229]:
data_new_r = dataise[['Internship_ID','Student_ID','Expected_Stipend','Minimum_Duration','Preferred_location','Is_Part_Time',
                      'Is_Shortlisted','source','Internship_Profile','Internship_Type','Internship_Location',
                      'Internship_category','No_of_openings','Stipend_Type','Stipend1','Internship_Duration.Months.','skill_compress_str',
                      'Institute_Category','Degree','Stream','Current_year','Year_of_graduation','PG_scale','UG_Scale',
                   'Performance_12th','Performance_10th','Freq','Experience_Type','avg_stipend','percentage_pg',
                   'percentage_ug','location_match','Stipend_low_match','Stipend_avg_match','student_stipend'
                  ]]
data_new = dataise[['Internship_ID','Student_ID','Expected_Stipend','Minimum_Duration','Is_Part_Time',
                      'Is_Shortlisted','source','Internship_Type',
                      'Internship_category','No_of_openings','Stipend_Type','Stipend1','Internship_Duration.Months.',
                      'Institute_Category','Current_year','Year_of_graduation','PG_scale','UG_Scale',
                   'Performance_12th','Performance_10th','Freq','Experience_Type','avg_stipend','percentage_pg',
                   'percentage_ug','location_match','Stipend_low_match','Stipend_avg_match','student_stipend'
                  ]]

In [230]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_to_encod_r = ['Preferred_location','Is_Part_Time','Internship_Profile','Internship_Type','Internship_Location','Internship_category',
                 'Stipend_Type','Institute_Category','skill_compress_str','Institute_Category','Degree','Stream','Year_of_graduation',
                 'PG_scale','UG_Scale','Experience_Type','location_match','Stipend_low_match','Stipend_avg_match','Current_year']
var_to_encode = ['Is_Part_Time','Internship_Type','Internship_category',
                 'Stipend_Type','Institute_Category','Institute_Category','Year_of_graduation',
                 'PG_scale','UG_Scale','Experience_Type','location_match','Stipend_low_match','Stipend_avg_match','Current_year']
for col in var_to_encode:
    data_new[col] = le.fit_transform(data_new[col])

C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [231]:
data_new = pd.get_dummies(data_new, columns=var_to_encode)
#data_new.columns
#train = data_new.loc[data_new['source'] == 'train']
#test = data_new.loc[data_new['source']=='test']
#train.drop('source',axis=1,inplace=True)
#predictors = [x for x in train.columns if x not in ['Internship_ID','Student_ID','Is_Shortlisted']]
#gbm0 = GradientBoostingClassifier(random_state=143)
#modelfit(gbm0,train,test,predictors)

MemoryError: 

In [105]:
test['Predictions'] = gbm0.predict(test[predictors])

C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [112]:
test.to_csv("finally2.csv")

In [111]:
#from sklearn.linear_model import LogisticRegression
#logreg = LogisticRegression()
#logreg.fit(train[predictors],train['Is_Shortlisted'])
test['predictionglm']=logreg.predict(test[predictors])

C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [115]:
data_new_r.to_csv("r_data_new.csv")

In [45]:
#student_exp['Start.Date'] = pd.to_datetime(student_exp['Start.Date'], format="%m/%d/%Y")
#from datetime import datetime
#date_format = "%m/%d/%Y"
#student_exp['Start.Date'] datetime.strptime('8/18/2008', date_format)
#b = datetime.strptime('9/26/2008', date_format)
#student_exp['exp_days'] = np.where(student_exp['Start.Date'].isnull(),0, np.where(student_exp['End.Date'].isnull(),-1,
#                                                                                 student_exp['End.Date']-student_exp['Start.Date']))

In [29]:
#dataise['degree_base'] = 'Others'
#words = pd.DataFrame({'a': [], 'b' : []})
#words['a'] = ['undergraduate program','djilsfj;kjello','B.E','.abkeelkjdaskf']
#words['b'] = ['a','a','a','a']
dataise['degree_base'] = 'empty'
#for i in range(4):
#    words['b'][i] = np.where(re.search(r'^(u|b|B|U)',words['a'][i]),'UG','No')
for i in range(10):
    dataise['Degree'][i] = str(dataise['Degree'][i])
    dataise['degree_base'][i] = np.where(re.search(r'^(u|b|B|U)',dataise['Degree'][i]),'UG',np.where(
            re.search(r'^(m|p|M|P)',dataise['Degree'][i]),'PG','Others'))


C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
def common_elements(list1, list2):
    a =  [element for element in list1 if element in list2]
    return len(a)
dataise['profile_common'] = 0
dataise['skill_common'] = 0
dataise['skill_common2'] = 0
for i in range(len(dataise)):
    dataise['profile_common'][i] = common_elements(dataise['Internship_Profile'][i],dataise['Profile'][i])
    dataise['skill_common'][i] = common_elements(dataise['Skills_required'][i],dataise['Profile'][i])
    dataise['skill_common2'][i] = common_elements(dataise['skill_compress_str'][i],dataise['Profile'][i])

In [16]:
dataise.dtypes

Internship_ID                    int64
Student_ID                       int64
Earliest_Start_Date             object
Expected_Stipend                object
Minimum_Duration                 int64
Preferred_location              object
Is_Part_Time                     int64
Is_Shortlisted                 float64
source                          object
Internship_Profile              object
Skills_required                 object
Internship_Type                 object
Internship_Location             object
Internship_category             object
No_of_openings                   int64
Stipend_Type                    object
Stipend1                       float64
Internship_deadline             object
Start_Date                      object
Internship_Duration.Months.      int64
skill_compress_str              object
Institute_Category              object
Degree                          object
Stream                          object
Current_year                    object
Year_of_graduation       

In [232]:
data_new.to_csv("glm_data.csv")

In [24]:
len(dataise)

300010

In [28]:
dataise['degree_base'].value_counts()

TypeError: unhashable type: 'numpy.ndarray'

In [30]:
dataise[['Degree','degree_base']]

,Degree,degree_base
0,B.E,UG
1,B.E,UG
2,B.E,UG
3,B.E,UG
4,B.E,UG
5,B.E,UG
6,Bachelor of Economics (B.Econ.) (Hons.),UG
7,MCA,PG
8,MCA,PG
9,MCA,PG
